In [1]:
from netCDF4 import Dataset as NetCDFFile 
import numpy as np
import os

In [2]:
import pandas as pd
import plotly.express as px


def display_map_sum(var_name1, var_name2,  day, dark=True, day2=None, daySum=None, difference = False, colourScale=None, Rad=3, invert=False, outputFolder = 'C:\CWatM_output', outputFolder2 = 'C:\CWatM_output', colourRange=None, compare=None):

  # The function displays the map of *var_name* on *day*.
  # If day2 != None, this displayed the difference between *day* and *day2*.
  print('testing')
  print('difference', difference)
  print(var_name1)
  print('hello hello')
  print(var_name2)
  for var_name in [var_name1, var_name2]:
        
    if 'month' not in var_name and 'annual' not in var_name :
    #if var_name == 'discharge' or var_name == 'head_development' or var_name == 'act_nonIrrWithdrawalGW' or var_name=='totalET' or var_name == 'act_nonIrrWithdrawalSW':
      nc = NetCDFFile(outputFolder + '/'+var_name +'_daily.nc')
    else:
      nc = NetCDFFile(outputFolder + '/'+var_name + '.nc')

    lat = nc.variables['lat'][:]
    lon = nc.variables['lon'][:]
    time = nc.variables['time'][:]

    if day2 != None:
      print('Showing difference between timestep '+str(day)+ ' and timestep'+str(day2))
      t2 = nc.variables[var_name][day,:,:] - nc.variables[var_name][day2,:,:]
    elif daySum != None: 
      print('Showing the sum of '+var_name+ ' over the simulation period. Unit are in metres (depth).')
      t2 = nc.variables[var_name][0,:,:]
      #t2 = nc.variables[var_name][day,:,:]
      #for d in range(day, daySum):
      print('number of days or months', len(nc.variables[var_name][:,0,0]))
      for d in range(len(nc.variables[var_name][:,0,0])):
        t2 += nc.variables[var_name][d,:,:]
    else:
      t2 = nc.variables[var_name][day,:,:]
    
    df = {'Latitude':[], 'Longitude':[], var_name:[], 'radius':[]}
    for i in range(len(t2[:,0])):
      for j in range(len(t2[0,:])):

        if t2[i,j]!=1200:
          df['Latitude'].append(lat[i])
          df['Longitude'].append(lon[j])

          if var_name == 'head_development' and invert == True:
            df[var_name].append(100-t2[i,j])
          else:
            #df[var_name].append(min(1200,t2[i,j])) #Notice the minimum
            df[var_name].append(t2[i,j])

          #df[var_name].append(max(min(1200,t2[i,j]),0.3)) #Notice the minimum
          #df[var_name].append(t2[i,j]) #Notice the minimum
          df['radius'].append(np.maximum(1, t2[i,j]/100)) #Notice the minimum
          #df_yield['Yield'].append(t2[i,j])
    print('in difference1')
    if var_name==var_name1:
        difference = df[var_name].copy()
     
    elif difference == False:
        difference = [difference[i] +  df[var_name][i] for i in range(len(difference))]
    
    elif difference == True:
        difference = [difference[i] - df[var_name][i] for i in range(len(difference))]

  df['difference'] = difference
  if compare==True:
    var_name = 'difference'


  if colourScale == None:
    colourScale="YlGnBu"

  if var_name == 'discharge_monthtot':
    #fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.23, lon=74.8), range_color = [0,300], zoom=8.5, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
    fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.1375, lon=74.4625), range_color = [0,300], zoom=8.8, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
 
  else:
    fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z='difference', radius=Rad, center=dict(lat=18.1375, lon=74.4625), zoom=7, range_color = colourRange, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 

  # Bhima zoom = 6.9
  fig.update_layout(mapbox_style="satellite", mapbox_accesstoken='pk.eyJ1Ijoic21pbG92aWMiLCJhIjoiY2tjamVwYWF2MThncTJ4cXFxbWVjdzY2diJ9.u_haNh8K1p-9oATjHgQ6LA')

  fig.update_layout(
      title_text = var_name,
      margin=dict(l=5, r=5, t=30, b=10),
      paper_bgcolor='rgba(0,0,0,0)',
      font=dict( color="white"))

  if dark != True:
      fig.update_layout(
          font=dict( color="black"))

  fig.show()

In [3]:
import pandas as pd
import plotly.express as px

# np.minimum(np.maximum(df[var_name], 1),10)
def display_map_sum_differentRuns(var_name,  day, dark=True, day2=None, daySum=None, calcDifference = False, colourScale=None, Rad=3, invert=False, outputFolder = 'C:\CWatM_output', outputFolder2 = 'C:\CWatM_output_Examples\CWatM_output_oneYear_reduceIrrigation', colourRange=None, compare=None):

  # The function displays the map of *var_name* on *day*.
  # If day2 != None, this displayed the difference between *day* and *day2*.
  print('testing')
  #print('difference', difference)
  print(var_name)

  for output in [outputFolder, outputFolder2]:
        
    if 'month' not in var_name and 'annual' not in var_name :
    #if var_name == 'discharge' or var_name == 'head_development' or var_name == 'act_nonIrrWithdrawalGW' or var_name=='totalET' or var_name == 'act_nonIrrWithdrawalSW':
      nc = NetCDFFile(output + '/'+var_name +'_daily.nc')
    else:
      nc = NetCDFFile(output + '/'+var_name + '.nc')

    lat = nc.variables['lat'][:]
    lon = nc.variables['lon'][:]
    time = nc.variables['time'][:]

    if day2 != None:
      print('Showing difference between timestep '+str(day)+ ' and timestep'+str(day2))
      t2 = nc.variables[var_name][day,:,:] - nc.variables[var_name][day2,:,:]
    elif daySum != None: 
      print('Showing the sum of '+var_name+ ' over the simulation period. Unit are in metres (depth).')
      t2 = nc.variables[var_name][0,:,:]
      #t2 = nc.variables[var_name][day,:,:]
      #for d in range(day, daySum):
      print('number of days or months', len(nc.variables[var_name][:,0,0]))
      for d in range(len(nc.variables[var_name][:,0,0])):
        t2 += nc.variables[var_name][d,:,:]
    else:
      t2 = nc.variables[var_name][day,:,:]
    
    df = {'Latitude':[], 'Longitude':[], var_name:[], 'radius':[]}
    for i in range(len(t2[:,0])):
      for j in range(len(t2[0,:])):

        if t2[i,j]!=1200:
          df['Latitude'].append(lat[i])
          df['Longitude'].append(lon[j])

          if var_name == 'head_development' and invert == True:
            df[var_name].append(100-t2[i,j])
          else:
            #df[var_name].append(min(1200,t2[i,j])) #Notice the minimum
            df[var_name].append(t2[i,j])

          #df[var_name].append(max(min(1200,t2[i,j]),0.3)) #Notice the minimum
          #df[var_name].append(t2[i,j]) #Notice the minimum
          df['radius'].append(np.maximum(1, t2[i,j]/100)) #Notice the minimum
          #df_yield['Yield'].append(t2[i,j])

    if output==outputFolder:
        difference = df[var_name].copy()
     
    elif calcDifference == False:
        print('not in difference')
        difference = [difference[i] +  df[var_name][i] for i in range(len(difference))]
    
    elif calcDifference == True:
        print('in real difference')
        difference = [difference[i] - df[var_name][i] for i in range(len(difference))]

  df['difference'] = difference
  if compare==True:
    var_name = 'difference'


  if colourScale == None:
    colourScale="YlGnBu"

  if var_name == 'discharge_monthtot':
    #fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.23, lon=74.8), range_color = [0,300], zoom=8.5, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
    fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.1375, lon=74.4625), range_color = [0,300], zoom=8.8, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
 
  else:
    fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z='difference', radius=Rad, center=dict(lat=18.1375, lon=74.4625), zoom=7, range_color = colourRange, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 

  # Bhima zoom = 6.9
  fig.update_layout(mapbox_style="satellite", mapbox_accesstoken='pk.eyJ1Ijoic21pbG92aWMiLCJhIjoiY2tjamVwYWF2MThncTJ4cXFxbWVjdzY2diJ9.u_haNh8K1p-9oATjHgQ6LA')

  fig.update_layout(
      title_text = var_name,
      margin=dict(l=5, r=5, t=30, b=10),
      paper_bgcolor='rgba(0,0,0,0)',
      font=dict( color="white"))

  if dark != True:
      fig.update_layout(
          font=dict( color="black"))

  fig.show()

In [4]:
import pandas as pd
import plotly.express as px

# np.minimum(np.maximum(df[var_name], 1),10)
def display_map_noTime(var_name, day, day2=None, density=True, dark=True, daySum=None, colourScale=None, Rad=1, invert=False, outputFolder = 'C:\CWatM_output', colourRange=None, compare=None):

      # The function displays the map of *var_name* on *day*.
      # If day2 != None, this displayed the difference between *day* and *day2*.
    nc = NetCDFFile(outputFolder + '/'+var_name + '.nc')
    lat = nc.variables['lat'][:]
    lon = nc.variables['lon'][:]
    print(len(lon))
    t2 = nc.variables[var_name][:,:]

    df = {'Latitude':[], 'Longitude':[], var_name:[]}
    for i in range(len(t2[:,0])):
        for j in range(len(t2[0,:])):
            if t2[i,j]>0:
                df['Latitude'].append(lat[i])
                df['Longitude'].append(lon[j])
                #df[var_name].append(min(500000000,t2[i,j]))
                #df[var_name].append(t2[i,j])
                df[var_name].append(1)

    if colourScale == None:
        colourScale="YlGnBu"

    if density:
        fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.1375, lon=74.4625), zoom=6.9, range_color = colourRange, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
    else:
        fig = px.scatter_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', size=var_name, center=dict(lat=18.1375, lon=74.4625), zoom=6.9, range_color = [0,300], color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 

      # Bhima zoom = 6.9
    fig.update_layout(mapbox_style="satellite", mapbox_accesstoken='pk.eyJ1Ijoic21pbG92aWMiLCJhIjoiY2tjamVwYWF2MThncTJ4cXFxbWVjdzY2diJ9.u_haNh8K1p-9oATjHgQ6LA')

    fig.update_layout(
        title_text = var_name,
        margin=dict(l=5, r=5, t=30, b=10),
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict( color="white"))


    if dark != True:
        fig.update_layout(font=dict( color="black"))

    fig.show()

In [5]:
import pandas as pd
import plotly.express as px

# np.minimum(np.maximum(df[var_name], 1),10)
def display_map(var_name, day, day2=None, dark=True, daySum=None, colourScale=None, Rad=5, invert=False, outputFolder = 'C:\CWatM_output', colourRange=None, compare=None):

      # The function displays the map of *var_name* on *day*.
      # If day2 != None, this displayed the difference between *day* and *day2*.

      print(var_name)
      if 'totalend' in var_name or 'annual' in var_name:
            nc = NetCDFFile(outputFolder + '/'+var_name + '.nc')
      elif 'month' not in var_name:
      #if var_name == 'discharge' or var_name == 'head_development' or var_name == 'act_nonIrrWithdrawalGW' or var_name=='totalET' or var_name == 'act_nonIrrWithdrawalSW':
        nc = NetCDFFile(outputFolder + '/'+var_name +'_daily.nc')
      else:
        nc = NetCDFFile(outputFolder + '/'+var_name + '.nc')

      lat = nc.variables['lat'][:]
      lon = nc.variables['lon'][:]
      time = nc.variables['time'][:]

      if day2 != None:
        print('Showing difference between timestep '+str(day)+ ' and timestep'+str(day2))
        t2 = nc.variables[var_name][day,:,:] - nc.variables[var_name][day2,:,:]
      elif daySum != None: 
        print('Showing the sum of '+var_name+ ' over the simulation period. Units are in metres (depth).')
        t2 = nc.variables[var_name][0,:,:]
        #t2 = nc.variables[var_name][day,:,:]
        #for d in range(day, daySum):
        print('number of days', len(nc.variables[var_name][:,0,0]))
        for d in range(len(nc.variables[var_name][:,0,0])):
          t2 += nc.variables[var_name][d,:,:]
      else:
        t2 = nc.variables[var_name][day,:,:]

      df = {'Latitude':[], 'Longitude':[], var_name:[], 'radius':[]}
      for i in range(len(t2[:,0])):
        for j in range(len(t2[0,:])):

          if t2[i,j]>0:
            df['Latitude'].append(lat[i])
            df['Longitude'].append(lon[j])

            if var_name == 'head_development' and invert == True:
              df[var_name].append(100-t2[i,j])
            else:
              #df[var_name].append(min(1200,t2[i,j])) #Notice the minimum
              df[var_name].append(t2[i,j])

            #df[var_name].append(max(min(1200,t2[i,j]),0.3)) #Notice the minimum
            #df[var_name].append(t2[i,j]) #Notice the minimum
            df['radius'].append(np.maximum(1, t2[i,j]/100)) #Notice the minimum
            #df_yield['Yield'].append(t2[i,j])
      if colourScale == None:
        colourScale="YlGnBu"

      if var_name == 'discharge_monthtot':
        #fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.23, lon=74.8), range_color = [0,300], zoom=8.5, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
        fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.1375, lon=74.4625), range_color = [0,300], zoom=8.8, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
        
      else:
        fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.1375, lon=74.4625), zoom=6.9, range_color = colourRange, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 

      # Bhima zoom = 6.9
      fig.update_layout(mapbox_style="satellite", mapbox_accesstoken='pk.eyJ1Ijoic21pbG92aWMiLCJhIjoiY2tjamVwYWF2MThncTJ4cXFxbWVjdzY2diJ9.u_haNh8K1p-9oATjHgQ6LA')

      fig.update_layout(
          title_text = var_name,
          margin=dict(l=5, r=5, t=30, b=10),
          paper_bgcolor='rgba(0,0,0,0)',
          font=dict( color="white"))


      if dark != True:
          fig.update_layout(
              font=dict( color="black"))

      fig.show()

In [6]:
#display_map_noTime('head_development',  200, colourScale='Blues', Rad=5, dark=False, daySum = True, outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'
#display_map_noTime('frac_totalIrr_max_totalend',  -1, density=True, colourScale="agsunset", Rad=3, dark=False, daySum = True, outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'
#display_map('head_development_monthend', day=22, Rad=2.1,  colourScale ='RdYlBu_r',dark=False, colourRange= [0,20], outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'
#display_map_sum('fracVegCover[3]_monthend', 'fracVegCover[1]_monthend', day=1, Rad=2, colourScale ='BrBG', dark=False,outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'
#display_map('fracVegCover[3]_monthend, day=-1, Rad=2.1,  colourScale ='RdYlBu_r',dark=False, colourRange= [0,20], outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'
#display_map('discharge_annualtot', day=-1, Rad=2.1,  colourScale ='Blues',dark=False, colourRange= [0,100], outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'
#display_map_sum('fracVegCover[3]_monthend', 'fracVegCover[1]_monthend', day=1, difference = True, Rad=2, colourScale ='BrBG', dark=False,outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'

#display_map_sum_differentRuns('fracVegCover[3]_monthend', day=-1, calcDifference = True, Rad=2, colourScale ='RdBu_r', dark=False,outputFolder = 'C:\CWatM_output') #)2.6) #colourScale='balance_r'
#display_map_noTime('waterBodyID_totalend',  -1, density=True, colourScale="agsunset", Rad=3, dark=False, daySum = True, colourRange=[0,1], outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'



In [40]:
display_map('discharge', day=-1, Rad=6, daySum=True, colourRange= [0,1000000], colourScale ='Blues', outputFolder = r'C:\CWatM_output_Examples\Danube_01012000_01012006') #)2.6) #colourScale='balance_r'


discharge
Showing the sum of discharge over the simulation period. Units are in metres (depth).
number of days 2193
